# VacationPy
---

## Starter Code to Import Libraries and Load the Weather and Coordinates Data

In [43]:
# Dependencies and Setup
import hvplot.pandas
import pandas as pd
import requests

# Import API key
from api_keys import geoapify_key

In [44]:
# Load the CSV file created in Part 1 into a Pandas DataFrame
city_data_df = pd.read_csv("output_data/cities.csv")

# Display sample data
city_data_df.head()

,City_ID,City,Lat,Lng,Max Temp,Humidity,Cloudiness,Wind Speed,Country,Date
0,0,shingu,33.7333,135.9833,27.22,78,2,0.43,JP,1726180581
1,1,grytviken,-54.2811,-36.5092,-0.06,85,54,5.56,GS,1726180582
2,2,bredasdorp,-34.5322,20.0403,9.38,92,98,2.15,ZA,1726180583
3,3,albany,42.6001,-73.9662,22.59,69,13,0.95,US,1726180501
4,4,isafjordur,66.0755,-23.1240,2.25,81,42,2.18,IS,1726180586


---

### Step 1: Create a map that displays a point for every city in the `city_data_df` DataFrame. The size of the point should be the humidity in each city.

In [49]:
%%capture --no-display

# Configure the map plot
map_plot = city_data_df.hvplot.points(
    "Lng", 
    "Lat", 
    geo = True, 
    tiles = "OSM", 
    frame_width = 700, 
    frame_height = 500, 
    size = "Humidity", 
    scale = 0.75, 
    alpha = 0.60,
    color = "City")

# Display the map
map_plot

:Overlay
   .WMTS.I   :WMTS   [Longitude,Latitude]
   .Points.I :Points   [Lng,Lat]   (City,Humidity)

### Step 2: Narrow down the `city_data_df` DataFrame to find your ideal weather condition

In [66]:
# Narrow down cities that fit criteria and drop any results with null values
ideal_weather_df = city_data_df[
    # Temperature between 18 and 29 degrees C
    (city_data_df["Max Temp"] >= 18) &
    (city_data_df["Max Temp"] <= 29) &
    # Wind Speed less than 3.5 m/s
    (city_data_df["Wind Speed"] <= 3.5) &
    # Cloudiness less than 50%
    (city_data_df["Cloudiness"] <= 50) &
    # Humidity less than 70%
    (city_data_df["Humidity"] <= 70)]


# Drop any rows with null values
ideal_weather_cleaned = ideal_weather_df.dropna(how='any')

# Display sample data
ideal_weather_cleaned.head()

,City_ID,City,Lat,Lng,Max Temp,Humidity,Cloudiness,Wind Speed,Country,Date
3,3,albany,42.6001,-73.9662,22.59,69,13,0.95,US,1726180501
8,8,tejen,37.3834,60.5054,19.92,21,0,1.00,TM,1726180590
51,51,yeppoon,-23.1333,150.7333,22.60,64,43,3.35,AU,1726180641
104,104,elvas,38.8815,-7.1628,22.03,62,7,3.16,PT,1726180703
115,115,jamestown,42.0970,-79.2353,27.23,44,0,1.34,US,1726180553


### Step 3: Create a new DataFrame called `hotel_df`.

In [67]:
# Use the Pandas copy function to create DataFrame called hotel_df to store the city, country, coordinates, and humidity
hotel_df = ideal_weather_cleaned[[
    "City", 
    "Country", 
    "Lat", 
    "Lng", 
    "Humidity"]].copy()

# Add an empty column, "Hotel Name," to the DataFrame so you can store the hotel found using the Geoapify API
hotel_df["Hotel Name"] = ""

# Display sample data
hotel_df.head()

,City,Country,Lat,Lng,Humidity,Hotel Name
3,albany,US,42.6001,-73.9662,69,
8,tejen,TM,37.3834,60.5054,21,
51,yeppoon,AU,-23.1333,150.7333,64,
104,elvas,PT,38.8815,-7.1628,62,
115,jamestown,US,42.0970,-79.2353,44,


### Step 4: For each city, use the Geoapify API to find the first hotel located within 10,000 metres of your coordinates.

In [70]:
# Set parameters to search for a hotel
radius = 10000
categories = "accommodation.hotel"
limit = 10
filters = []
bias = []

params = {
    "categories" : categories, 
    "limit" : limit, 
    "filter" : filters, 
    "bias" : bias, 
    "apiKey" : geoapify_key
}

# Print a message to follow up the hotel search
print("Starting hotel search")

# Iterate through the hotel_df DataFrame
for index, row in hotel_df.iterrows():
    
    # get latitude, longitude from the DataFrame
    longitude = float(row["Lng"])
    latitude = float(row["Lat"])

    # Add the current city's latitude and longitude to the params dictionary
    params["filter"] = f"circle:{longitude},{latitude},{radius}"
    params["bias"] = f"proximity:{longitude},{latitude}"

    # Set base URL
    base_url = "https://api.geoapify.com/v2/places"

    # Make an API request using the params dictionary
    name_address = requests.get(base_url, params = params)

    # Convert the API response to JSON format
    name_address = name_address.json()

    # Grab the first hotel from the results and store the name in the hotel_df DataFrame
    try:
        hotel_df.loc[index, "Hotel Name"] = name_address["features"][0]["properties"]["name"]
    except (KeyError, IndexError):
        # If no hotel is found, set the hotel name as "No hotel found".
        hotel_df.loc[index, "Hotel Name"] = "No hotel found"

    # Log the search results
    print(f"{hotel_df.loc[index, 'City']} - nearest hotel: {hotel_df.loc[index, 'Hotel Name']}")

# Display sample data
hotel_df

Starting hotel search
albany - nearest hotel: No hotel found
tejen - nearest hotel: Tejen Myhmanhanasy
yeppoon - nearest hotel: Blossom's on Seaspray
elvas - nearest hotel: Vila Galé Collection Elvas
jamestown - nearest hotel: DoubleTree Jamestown
filiatra - nearest hotel: Hotel Limenari
tukrah - nearest hotel: No hotel found
tagab - nearest hotel: No hotel found
varzea da palma - nearest hotel: No hotel found
saint-pierre - nearest hotel: Tropic Hotel
kalabo - nearest hotel: Golden Lodge Luxury Acomodation
fort st. john - nearest hotel: Fort St. John Motor Inn
ibia - nearest hotel: Hotel Paulista
el bayadh - nearest hotel: No hotel found
lambari - nearest hotel: Parque Hotel
robore - nearest hotel: Lajas
kotli - nearest hotel: No hotel found
chaman - nearest hotel: No hotel found
shetpe - nearest hotel: Балумит
hozat - nearest hotel: No hotel found
chibougamau - nearest hotel: No hotel found
sao vicente - nearest hotel: Pousada Vitória
hotan - nearest hotel: Hua Yi International Hotel

,City,Country,Lat,Lng,Humidity,Hotel Name
3,albany,US,42.6001,-73.9662,69,No hotel found
8,tejen,TM,37.3834,60.5054,21,Tejen Myhmanhanasy
51,yeppoon,AU,-23.1333,150.7333,64,Blossom's on Seaspray
104,elvas,PT,38.8815,-7.1628,62,Vila Galé Collection Elvas
115,jamestown,US,42.0970,-79.2353,44,DoubleTree Jamestown
159,filiatra,GR,37.1500,21.5833,67,Hotel Limenari
172,tukrah,LY,32.5341,20.5791,66,No hotel found
189,tagab,AF,34.8550,69.6492,23,No hotel found
194,varzea da palma,BR,-17.5981,-44.7308,29,No hotel found
221,saint-pierre,RE,-21.3393,55.4781,68,Tropic Hotel


### Step 5: Add the hotel name and the country as additional information in the hover message for each city in the map.

In [74]:
%%capture --no-display

# Configure the map plot
map_plot_hotels = hotel_df.hvplot.points(
    "Lng", 
    "Lat", 
    geo = True, 
    tiles = "OSM", 
    frame_width = 700, 
    frame_height = 500, 
    size = "Humidity", 
    scale = 0.95, 
    alpha = 0.60,
    color = "City", 
    hover_cols=["Lng", "Lat", "Humidity", "City", "Country", "Hotel Name"])

# Display the map
map_plot_hotels

:Overlay
   .WMTS.I   :WMTS   [Longitude,Latitude]
   .Points.I :Points   [Lng,Lat]   (City,Humidity,Country,Hotel Name)